In [1]:
#import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import config

In [2]:
#sqlalcehmy database connection
database = f'postgres://{config.user}:{config.db_password}@{config.host}/postgres'
engine = create_engine(database)

In [37]:
beer_db = pd.read_sql_query('''SELECT *
                    FROM beers as be
                    LEFT JOIN breweries as br
                    ON (be.brewery_id = br.brewery_id)
                    ;''',con=engine)
beer_db

,abv,ibu,beer_id,beer_name,style,style_group,brewery_id,ounces,brewery_id,brewery_name,city,state
0,0.061,60.0,1979,Bitter Bitch,American Pale Ale (APA),American Pale Ale (APA),177,12.0,177,18th Street Brewery,Gary,IN
1,0.099,92.0,1036,Lower De Boom,American Barleywine,other,368,8.4,368,21st Amendment Brewery,San Francisco,CA
2,0.079,45.0,1024,Fireside Chat,Winter Warmer,other,368,12.0,368,21st Amendment Brewery,San Francisco,CA
3,0.044,42.0,876,Bitter American,American Pale Ale (APA),American Pale Ale (APA),368,12.0,368,21st Amendment Brewery,San Francisco,CA
4,0.049,17.0,802,Hell or High Watermelon Wheat (2009),Fruit / Vegetable Beer,Fruit / Vegetable Beer,368,12.0,368,21st Amendment Brewery,San Francisco,CA
...,...,...,...,...,...,...,...,...,...,...,...,...
1321,0.077,30.0,1513,Lights Out Vanilla Cream Extra Stout,American Double / Imperial IPA,American Double / Imperial IPA,199,12.0,199,Worthy Brewing Company,Bend,OR
1322,0.069,69.0,1512,Worthy IPA (2013),American IPA,American IPA,199,12.0,199,Worthy Brewing Company,Bend,OR
1323,0.060,50.0,1511,Worthy Pale,American Pale Ale (APA),American Pale Ale (APA),199,12.0,199,Worthy Brewing Company,Bend,OR
1324,0.067,45.0,928,Belgorado,Belgian IPA,other,424,12.0,424,Wynkoop Brewing Company,Denver,CO


In [38]:
styles = (beer_db["style_group"].value_counts()[:20]).keys()

print(styles)

Index(['other', 'American IPA', 'American Pale Ale (APA)',
       'American Amber / Red Ale', 'American Double / Imperial IPA',
       'American Blonde Ale', 'American Pale Wheat Ale', 'American Brown Ale',
       'American Porter', 'Kölsch', 'Fruit / Vegetable Beer'],
      dtype='object')


In [39]:
# Seperate dataset into features (X) and target (y)

X = beer_db[['abv', 'ibu']] # Features
y = beer_db['style_group'] # Target

# splitting the dataset

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [40]:
# Creating scaler instance

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
y_train

246     American Amber / Red Ale
1242                       other
1246                American IPA
333     American Amber / Red Ale
192                        other
                  ...           
195                        other
608                        other
1325     American Pale Ale (APA)
927     American Amber / Red Ale
1181                       other
Name: style_group, Length: 994, dtype: object

In [41]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=128, random_state=48)
rfc.fit(X_train, y_train)

y_pred=rfc.predict(X_test)

In [42]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

                                precision    recall  f1-score   support

      American Amber / Red Ale       0.38      0.26      0.31        19
           American Blonde Ale       0.10      0.10      0.10        10
            American Brown Ale       0.14      0.07      0.10        14
American Double / Imperial IPA       0.75      0.75      0.75        20
                  American IPA       0.65      0.70      0.68        70
       American Pale Ale (APA)       0.42      0.45      0.43        40
       American Pale Wheat Ale       0.20      0.06      0.10        16
               American Porter       0.29      0.25      0.27         8
        Fruit / Vegetable Beer       0.00      0.00      0.00         7
                        Kölsch       0.14      0.20      0.17         5
                         other       0.50      0.55      0.53       123

                      accuracy                           0.48       332
                     macro avg       0.33      0.31      0.31 

NameError: name 'metrics' is not defined

In [43]:
tr_sc = pd.concat([X_train,y_train], axis=1)
tr_sc = tr_sc.sort_values(by=['abv', 'ibu'])

In [44]:
tr_sc

,abv,ibu,style_group
577,0.027,21.0,other
1317,0.028,15.0,other
1098,0.032,13.0,other
79,0.032,27.0,Kölsch
783,0.034,6.0,other
...,...,...,...
917,0.099,100.0,American Double / Imperial IPA
832,0.099,100.0,American Double / Imperial IPA
1188,0.099,101.0,American Double / Imperial IPA
995,0.099,111.0,American Double / Imperial IPA
